In [13]:
import os
import csv
from engine.files.files import File
from engine.audio import extract_peak
from engine.spectrograms import create_spectrogram
audioFile = "../data/audio/2015-06-08_ID8_0026.wav"
syllableFile = "../data/syllable/2015-06-08_ID8_0026.csv"
with open(syllableFile) as csv_file:
	syllableData = list(csv.reader(csv_file, delimiter=',', lineterminator="\n"))

syllableData = syllableData[1:]
audioData = File(audioFile)
destFile = File('../tmp/test.wav')
# os.mkdir(destFile.folder)
for line in syllableData:
	extract_peak(audioData.p, destFile.p, float(line[0]), .02, .02, 43.4)
	create_spectrogram(destFile.p, '../tmp/test.jpeg', sampling_rate=500000, x_pixels_per_sec=2000, window='Ham')
	break



Log for interactive will be written in log-2019-11-16_11-32-22


create metadata.json from the metdata csv (generated earlier with R). Include the peaks and do some typing on some values.

In [18]:
import csv
import json
from engine.helpers import write_file
from engine.files.files import File
with open('../data/metadata.csv') as csv_file:
	meta = list(csv.reader(csv_file, delimiter=',', lineterminator="\n"))

jsonData = dict()
header = meta[0]
meta = meta[1:]
for entry in meta:
	jsonEntry = dict((header[i], value) for i, value in enumerate(entry))
	jsonEntry['duration'] = float(jsonEntry['wav_length'])
	del jsonEntry['wav_length']
	jsonEntry['individual'] = int(jsonEntry['id'])
	jsonEntry['label'] = "A" + str(len(jsonData) + 1)

	with open(f'../data/' + jsonEntry['syllable']) as csv_file:
		peaks = list(csv.reader(csv_file, delimiter=',', lineterminator="\n"))

	peaksHeader = peaks[0]
	peaks = peaks[1:]
	jsonPeaks = []
	for peak in peaks:
		jsonPeak = dict((peaksHeader[i], value) for i, value in enumerate(peak))
		jsonPeak['timestamp'] = float(jsonPeak['timestamp'])
		jsonPeak['sequence'] = jsonPeak['sequence'].strip().upper()
		if jsonPeak['sequence'] in sequencesSorted:
			jsonPeaks.append(jsonPeak)


	jsonEntry['peaks'] = jsonPeaks
	jsonData[File(entry[7]).p.stem] = jsonEntry

write_file(f"../data/metadata.json", json.dumps(jsonData, indent=2))


In [5]:
from engine.metadata import metadata_db
mdb = metadata_db('metadata.json')
sequences = dict()
numSeq = 0
for stem in mdb.db:
	entry = mdb.db[stem]
	for peak in entry.peaks:
		numSeq = numSeq+1
		s = peak['sequence']
		if s in sequences:
			sequences[s] += 1
		else:
			sequences[s] = 1

In [17]:
sequencesSorted = sorted(dict(filter(lambda elem: elem[1] > 20, sequences.items())))

In [ ]:
from engine.files.files import File
from engine.spectrograms import create_spectrogram
from engine.metadata import metadata_db

mdb = metadata_db('metadata.json')
for stem in mdb.db:
	entry = mdb.db[stem]
	audioFile = File(f'../data/{entry.audio}')
	create_spectrogram(audioFile.p, f'../tmp/{stem}.jpeg', sampling_rate=500000, x_pixels_per_sec=1000, window='Ham', axes=True)



Log for interactive will be written in log-2019-11-14_11-52-11


create peak metadata, based on that create datasets


In [19]:
from engine.metadata import metadata_db
import csv
import datetime
numSeq = 0
phaseSepTime = {'ID8': datetime.date(2015, 6, 28), 'ID10': datetime.date(2015, 7, 10)}
sequenceMeta = dict()
mdb = metadata_db('metadata.json')
for stem in mdb.db:
	entry = mdb.db[stem]
	date = datetime.datetime.strptime(entry.date, '%Y-%m-%d').date()
	id = 'ID' + entry.id

	for peak in entry.peaks:
		numSeq = numSeq+1
		s = peak['sequence']

		if s not in sequenceMeta:
			sequenceMeta[s] = {'count':0, 'phase1':0, 'phase2':0}

		sequenceMeta[s]["count"] += 1
		if date < phaseSepTime[id]:
			sequenceMeta[s]["phase1"] += 1
		if date > phaseSepTime[id]:
			sequenceMeta[s]["phase2"] += 1


with open('../data/peaks.csv', 'w') as csvfile:
	writer = csv.DictWriter(csvfile, delimiter=',', lineterminator='\n', fieldnames=['sequence', 'count', 'phase1', 'phase2'])
	writer.writeheader()
	for key in sequenceMeta:
		writer.writerow({'sequence':key, **sequenceMeta[key]})



In [8]:
import math, datetime
from collections import defaultdict
from engine.metadata import metadata_db, filter_peaks
mdb = metadata_db('metadata.json')
min_occurrence = 20
validation_size = 4

sequence_count = defaultdict(lambda: 0)

for stem in mdb.db:
	entry = mdb.db[stem]
	for peak in entry.peaks:
		sequence_count[peak['sequence']] += 1

filtered_sequences = dict(filter(lambda elem: elem[1] > min_occurrence, sequence_count.items())).keys()
sequences = defaultdict(list)
validation_sequences = defaultdict(list)

for stem in mdb.db:
	entry = mdb.db[stem]
	time = datetime.datetime.strptime(entry.date + ' ' + entry.time, '%Y-%m-%d %H:%M:%S').timestamp()
	peaks = filter(lambda peak: peak['sequence'] in filtered_sequences, entry.peaks)
	entry.props['peaks'] = peaks
	for peak in peaks:
		sequences[peak['sequence']].append((peak['timestamp'] + time, entry.label))

for key in sequences:
	sequence_times = sequences[key]
	sequence_times.sort()
	length = len(sequence_times)
	nth = math.floor(length / validation_size)
	offset = (length % validation_size) + math.floor(nth / 2)
	validation_sequences[key] = sequence_times[offset::nth]

for stem in mdb.db:
	entry = mdb.db[stem]
	time = datetime.datetime.strptime(entry.date + ' ' + entry.time, '%Y-%m-%d %H:%M:%S').timestamp()
	entry.props['peaks'] = filter(lambda peak: (peak['timestamp'] + time, entry.label) in validation_sequences[peak['sequence']], entry.peaks)



In [ ]:
import os
import csv
from engine.files.files import File
from engine.audio import extract_peak
from engine.spectrograms import create_spectrogram
audioFile = "../data/audio/simpleCall/B2_C_B_Jt1_16_07_06_042.wav"

audioData = File(audioFile)
destFile = File('../tmp/test.wav')
# os.mkdir(destFile.folder)
for line in syllableData:
	extract_peak(audioData.p, destFile.p, float(line[0]), .02, .02, 43.4)
	create_spectrogram(destFile.p, '../tmp/test.jpeg', sampling_rate=500000, x_pixels_per_sec=2000, window='Ham')
	break
